# Dane obrobione

In [1]:
import pandas as pd

In [2]:
df_zapotrzebowanie = pd.read_parquet('data/obrobione_Zapotrzebowanie_2023.parquet')
df_agg_curves_fix1 = pd.read_parquet('data/obrobione_KrzyweZagregowaneFix1_2023.parquet')
df_ceny_rdn = pd.read_parquet('data/obrobione_Ceny_2023.parquet')

In [14]:
df_zapotrzebowanie

In [4]:
df_agg_curves_fix1

In [5]:
df_ceny_rdn

# Standardowe profile zużycia 

In [6]:
df = pd.read_excel('Załącznik_4_IRiESD_profile_2023.xlsx', sheet_name='G11')
df = df.drop(columns='2a') 

df = df.set_index('Data')
df = df.drop(columns=['Dzień'])
df = df.melt(ignore_index=False, var_name='Godzina', value_name='Zuzycie_MWh')

df['Godzina'] = df['Godzina'].astype(float)
df = df.reset_index(drop=False)
df['Data'] = df['Data'].dt.date

# wyrzucenie godziny 2 w dniu z 23 godzinami
df = df.dropna()

df['Data Godzina'] = pd.to_datetime(df['Data']) + pd.to_timedelta(df['Godzina'] - 1, unit='h')

df = df.set_index('Data Godzina').sort_index()
df = df.rename(columns={'Zuzycie_MWh': 'Zuzycie gd MWh'})
# df.to_excel('gospodarstwa_domowe_zuzycie_enea_melt_2023.xlsx', index=False)

In [7]:
df

Stworzenie godzinowego zużycia ee dla gospodarstw domowych w całym kraju

In [10]:
# dane z Eurostatu w GWh
df['Roczne zuzycie gd MWh'] = 29774.5 * 1000

df['Zuzycie gd %'] = df['Zuzycie gd MWh'] / df['Zuzycie gd MWh'].sum()
# zamodelowanie godzinowego zużycia ee przez gospodarstwa domowe
df['Godzinowe zuzycie gd MWh'] = round(df['Roczne zuzycie gd MWh'] * df['Zuzycie gd %'], 4)
df = df.drop(columns=['Roczne zuzycie gd MWh', 'Zuzycie gd %', 'Zuzycie gd MWh'])

# Dodanie krajowego zapotrzeobwania

In [12]:
from config import mongo_connect_config_pse_pk5
from utils.from_mongodb import MongoData

In [13]:
pse = MongoData(mongo_connect_config_pse_pk5)
df_pse = pse.load_mongo_document_to_df()
df_pse.to_parquet('data/surowe_ZapotrzebowaniePSE_2023.parquet')

In [93]:
df_pse['Data Godzina'] = pd.to_datetime(df_pse['date_cet']) + pd.to_timedelta(df_pse['hour'] - 1, unit='h')
df_pse = df_pse.set_index('Data Godzina')
df_pse = df_pse[['prognozowane_zapotrzebowanie_sieci[MW]']].copy()
df_pse = df_pse.rename(columns={'prognozowane_zapotrzebowanie_sieci[MW]': 'Godzinowe zuzycie krajowe MWh'})

df = df.merge(df_pse, left_index=True, right_index=True)

In [95]:
df.to_parquet('data/obrobione_Zapotrzebowanie_2023.parquet')

# Krzywe zagregowane RDN

Wczytanie krzywych DAMC

In [16]:
from config import mongo_connect_config_krzywe_zagregowane

In [8]:
curves = MongoData(mongo_connect_config_krzywe_zagregowane)
df_agg_curves = curves.load_mongo_document_to_df()
df_agg_curves.to_parquet('data/surowe_KrzyweZagregowaneDAMC_2023.parquet')

In [17]:
df_agg_curves = pd.read_parquet('data/surowe_KrzyweZagregowaneDAMC_2023.parquet')

Wczytanie krzywych Fix1

In [102]:
from config import mongo_connect_config_krzywe_zagregowane_fix1

In [2]:
curves = MongoData(mongo_connect_config_krzywe_zagregowane_fix1)
df_agg_curves_fix1 = curves.load_mongo_document_to_df()
df_agg_curves_fix1.to_parquet('data/surowe_KrzyweZagregowaneFix1_2023.parquet')

In [98]:
df_agg_curves_fix1 = pd.read_parquet('data/surowe_KrzyweZagregowaneFix1_2023.parquet')

Braki w danych

In [99]:
def check_missing_data(df, config):
    return pd.date_range(start = config.START, end = config.END).difference(df['date_cet'])

In [100]:
# fix1
check_missing_data(df_agg_curves_fix1, mongo_connect_config_krzywe_zagregowane_fix1)

In [105]:
# damc
check_missing_data(df_agg_curves, mongo_connect_config_krzywe_zagregowane)

In [106]:
print(f"""
Ilość danych fix {len(df_agg_curves_fix1)},
Ilość danych damc {len(df_agg_curves)}
"""
)


## Krzywe Fix1 - przetworzenie danych

Znalezienie punktu przecięcia krzywych

In [18]:
import numpy as np

df_agg_curves = pd.read_parquet('data/surowe_KrzyweZagregowaneFix1_2023.parquet')
curr = 'PLN/MWh'

df_ = df_agg_curves[
    (df_agg_curves['date_cet'] == '2023-11-18')
    & (df_agg_curves['interval_index'] == 1)
].copy()

def _rename_cols(curr, df_):
    df_dict = {}
    for type_ in ['sell', 'buy']:
        df_dict[type_] = df_[[f'aggt_{type_}_vol[MWh]', f'{type_}_price[{curr}]']]
        df_dict[type_] = df_dict[type_].rename(columns={
            f'aggt_{type_}_vol[MWh]': 'aggt_vol[MWh]',
            f'{type_}_price[{curr}]': f'price[{curr}]'},
        )
        df_dict[type_]['type'] = type_

    df_ = pd.concat([df_dict['sell'], df_dict['buy']])
    df_ = df_.reset_index(drop=True)

    return df_

df_ = _rename_cols(curr, df_)
df_ = df_.sort_values(by=['aggt_vol[MWh]']).reset_index(drop=True)
df_[f'price_sell[{curr}]'] = df_.apply(lambda x: x[f'price[{curr}]'] if x['type'] == 'sell' else np.nan, axis=1)
df_[f'price_buy[{curr}]'] = df_.apply(lambda x: x[f'price[{curr}]'] if x['type'] == 'buy' else np.nan, axis=1)
df_[[f'price_buy[{curr}]', f'price_sell[{curr}]']] = df_[[f'price_buy[{curr}]', f'price_sell[{curr}]']].bfill()
df_row_to_append = df_[df_[f'price_buy[{curr}]'] > df_[f'price_sell[{curr}]']][-1:]
df_row_to_append

In [19]:
df_ceny_rdn[df_ceny_rdn.index == '2023-11-18']

In [125]:
df_agg_curves_fix1 = pd.read_parquet('data/surowe_KrzyweZagregowaneFix1_2023.parquet')

df_agg_curves_fix1['Godzina'] = df_agg_curves_fix1['interval_index']
# wyrzucenie 25 godziny
df_agg_curves_fix1.drop(index=df_agg_curves_fix1[df_agg_curves_fix1['Godzina'] == 25].index, inplace=True)
df_agg_curves_fix1['Data Godzina'] = pd.to_datetime(df_agg_curves_fix1['date_cet']) + pd.to_timedelta(df_agg_curves_fix1['interval_index'] - 1, unit='h')
df_agg_curves_fix1 = df_agg_curves_fix1.rename(columns={'date_cet': 'Data'})

df_agg_curves_fix1 = df_agg_curves_fix1.set_index('Data Godzina')

df_curves_sell = df_agg_curves_fix1[
    ['Data', 'Godzina', 'aggt_sell_vol[MWh]', 'sell_price[PLN/MWh]']].copy()

df_curves_sell['Strona'] = 'sell'
df_curves_sell.rename(columns={'aggt_sell_vol[MWh]': 'Wolumen oferty MWh', 'sell_price[PLN/MWh]': 'Cena oferty PLN/MWh'},
                      inplace=True)

df_curves_buy = df_agg_curves_fix1[
    ['Data', 'Godzina', 'aggt_buy_vol[MWh]', 'buy_price[PLN/MWh]']].copy()
df_curves_buy['Strona'] = 'buy'
df_curves_buy.rename(columns={'aggt_buy_vol[MWh]': 'Wolumen oferty MWh', 'buy_price[PLN/MWh]': 'Cena oferty PLN/MWh'},
                     inplace=True)

df_curves = pd.concat([df_curves_buy, df_curves_sell])
df_curves.to_parquet('data/obrobione_KrzyweZagregowaneFix1_2023.parquet')

# Ceny RDN

In [126]:
from config import mongo_connect_config_ceny_fix1
from utils.from_mongodb import MongoData

In [ ]:
ceny = MongoData(mongo_connect_config_ceny_fix1)
df_ceny_rdn = ceny.load_mongo_document_to_df()
df_ceny_rdn.to_parquet('data/surowe_Ceny_2023.parquet')

In [128]:
df_ceny_rdn = pd.read_parquet('data/surowe_Ceny_2023.parquet')

In [129]:
df_ceny_rdn['Data Godzina'] = pd.to_datetime(df_ceny_rdn['date_cet']) + pd.to_timedelta(df_ceny_rdn['hour'] - 1, unit='h')
df_ceny_rdn = df_ceny_rdn.set_index('Data Godzina')

df_ceny_rdn = df_ceny_rdn.drop(columns=['interval_index', 'interval_type', 'wolumen_fixingu[MWh]']).copy()
df_ceny_rdn.rename(columns={'date_cet': 'Data', 'hour': 'Godzina', 'cena_fixingu[PLN/MWh]': 'Cena PLN/MWh'}, inplace=True)

In [131]:
df_ceny_rdn.to_parquet('data/obrobione_Ceny_2023.parquet')

In [30]:
df_ceny_rdn

In [33]:
df_agg_curves_fix1

In [44]:
df_sell = df_agg_curves_fix1[
    (df_agg_curves_fix1.index == '2023-01-01') &
    (df_agg_curves_fix1['Strona'] == 'sell')].sort_values('Cena oferty PLN/MWh')

df_buy = df_agg_curves_fix1[
    (df_agg_curves_fix1.index == '2023-01-01') &
    (df_agg_curves_fix1['Strona'] == 'buy')].sort_values('Cena oferty PLN/MWh')

In [46]:
df_buy